To measure PP1 intensities 

In [1]:
# import necessary modules
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import skimage.filters as filters
import skimage.color as color
from PIL import Image
import pandas as pd
from skimage.util import img_as_float
from skimage.color import label2rgb, rgb2gray,gray2rgb
from skimage import segmentation,morphology,feature,measure,filters
from scipy import ndimage as ndi
import glob
from natsort import natsorted 
import os
import glob
import re
import random

In [2]:
# import function file
%run functions-Method1.ipynb
random.seed(3)

In [3]:
# path to the local folder for the notebooks
# change path to your folders accordingly
BASE_DIR="/Users/Trupti/01-LIDo/02-VijiProject/ImageAnalysis/" 
#path to the image folder. # change the name of the folder for different dataset
FOLDER="set2_pp1"
IMAGE_DIR=BASE_DIR + "data/XMovie/PP1/" + FOLDER + "/"
CSV_DIR = BASE_DIR + "AnalysisMethods/AnalysisResults/XMovie/PP1/" + FOLDER +"/" + "csvs/"


#### A List for the Images Folders

In [4]:
dmso=[]
cdki=[]
dmso_path = IMAGE_DIR + 'dmso'
cdki_path=IMAGE_DIR + 'cdki'

for folder in glob.glob(dmso_path + '/*'):
    dmso.append(dmso_path + '/' + folder.split('/')[-1].split('.')[0])
for folder in glob.glob(cdki_path + '/*'):
    cdki.append(cdki_path + '/' + folder.split('/')[-1].split('.')[0])     


In [5]:
# For DMSO

for i in range(len(dmso)):
    
    df_nuf2,df_pp1=method_pp1(dmso[i],FOLDER) #,nuf2_cyto,pp1_cyto,nuf2_bg,pp1_bg
    
    df=pd.merge(df_nuf2, df_pp1,  how='outer',on=['label','area']
                                                       ,suffixes=('_nuf2_CFP', '_YFP_pp1'))
    #df['nuf2_CFP_cyto']=nuf2_cyto
    #df['YFP_pp1_cyto']=pp1_cyto
    #df['nuf2_CFP_bg']=nuf2_bg
    #df['YFP_pp1_bg']=pp1_bg
    
    directory=dmso[i].split('/')[-1].split('.')[0] # to create a folder per experiment to save csvs
    
    path = CSV_DIR + 'dmso/' + directory
    try:
        os.makedirs(path)
    except FileExistsError:
    # directory already exists
        pass
    check_nan_in_df = df.isnull().values.any()
   
    if check_nan_in_df >0:
        df=df.dropna()
        print("NaNs removed from %s " %( directory))
    
    
  
    df.to_csv(path + '/' + 'particles'+'.csv') 

   
   

<Figure size 432x288 with 0 Axes>

In [6]:
# For CDKi

for i in range(len(cdki)):
    
    df_nuf2,df_pp1=method_pp1(cdki[i],FOLDER)#,nuf2_cyto_value,pp1_cyto_value,nuf2_bg,pp1_bg
    
    df=pd.merge(df_nuf2, df_pp1,  how='outer',on=['label','area']
                                                       ,suffixes=('_nuf2_CFP', '_YFP_pp1'))
    #df['nuf2_CFP_cyto']=nuf2_cyto_value
    #df['YFP_pp1_cyto']=pp1_cyto_value
    #df['nuf2_CFP_bg']=nuf2_bg
    #df['YFP_pp1_bg']=pp1_bg
    directory=cdki[i].split('/')[-1].split('.')[0] # to create a folder per experiment to save csvs
    path = CSV_DIR + 'cdki/' + directory
    try:
        os.makedirs(path)
    except FileExistsError:
    # directory already exists
        pass
    
    check_nan_in_df = df.isnull().values.any()
    if check_nan_in_df >0:
        df=df.dropna()
        print("NaNs removed from %s " %( directory))
    df.to_csv(path + '/' + 'particles'+'.csv') 
   
  

<Figure size 432x288 with 0 Axes>

### Plots

In [7]:
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook",font_scale=1, rc={"lines.linewidth":2.5})

# path to save graphs
GRAPH_DIR=BASE_DIR + "AnalysisMethods/AnalysisResults/XMovie/" + FOLDER + "/" + "graphs/"


In [8]:
# plot graphs 
# dmso vs cdki
# The paths are for the generated csv files
dmso_csv=[]
cdki_csv=[]
dmso_path = CSV_DIR + 'dmso'
cdki_path=CSV_DIR + 'cdki'

for folder in natsorted(glob.glob(dmso_path + '/*')):
    dmso_csv.append(dmso_path + '/' + folder.split('/')[-1].split('.')[0])
    
for folder in natsorted(glob.glob(cdki_path + '/*')):
    cdki_csv.append(cdki_path + '/' + folder.split('/')[-1].split('.')[0])  


In [9]:
# read cdki csvs to a dataframe
cell_count=0
df_graph_cdki=pd.DataFrame()
for i in range(len(cdki_csv)):
    df_cdki=pd.read_csv(cdki_csv[i]+'/'+'Particles.csv')
    df_cdki['YFP_pp1@kt/YFP_pp1@cyto']= df_cdki['mean_intensity_YFP_pp1']/df_cdki['YFP_pp1_cyto']
    df_cdki['nuf2_CFP@kt/nuf2_CFP@cyto']= df_cdki['mean_intensity_nuf2_CFP']/df_cdki['nuf2_CFP_cyto']
    df_cdki['YFP_pp1@kt/YFP_pp1@cyto:nuf2_CFP@kt/nuf2_CFP@cyto']= df_cdki['YFP_pp1@kt/YFP_pp1@cyto'] / df_cdki['nuf2_CFP@kt/nuf2_CFP@cyto']
    df_cdki['condition']='cdki'
    df_cdki['cell']=FOLDER+"_"+'cdki_cell_'+str(cell_count)
    expt_name=re.sub(r'^.+/([^/]+)$', r'\1', cdki_csv[i])
    df_cdki['Experiment']= expt_name#'cdki_%d'%i
    fname=FOLDER +'cdki_'+ str(i)
    df_cdki_final=outliers_pp1(df_cdki)
    df_cdki_final.to_csv(BASE_DIR  + 'GraphAnalysis/PP1/'+FOLDER + '/' + fname +'.csv')
    df_graph_cdki=pd.concat([df_graph_cdki,df_cdki_final],ignore_index=True)
    cell_count+=1
df_graph_cdki = df_graph_cdki.drop(columns=["Unnamed: 0"], axis=1)
df_graph_cdki.to_csv(BASE_DIR  + 'GraphAnalysis/PP1/'+FOLDER+'/' +FOLDER+ "_" +'cdki_pp1_graphs'+'.csv') 
len(df_graph_cdki)

489

In [10]:
# read dmso csvs to a dataframe
cell_count=0
df_graph_dmso=pd.DataFrame()
for i in range(len(dmso_csv)):
    df_dmso=pd.read_csv(dmso_csv[i]+'/'+'Particles.csv')
    df_dmso['YFP_pp1@kt/YFP_pp1@cyto']= df_dmso['mean_intensity_YFP_pp1']/df_dmso['YFP_pp1_cyto']
    df_dmso['nuf2_CFP@kt/nuf2_CFP@cyto']= df_dmso['mean_intensity_nuf2_CFP']/df_dmso['nuf2_CFP_cyto']
    df_dmso['YFP_pp1@kt/YFP_pp1@cyto:nuf2_CFP@kt/nuf2_CFP@cyto']= df_dmso['YFP_pp1@kt/YFP_pp1@cyto'] / df_dmso['nuf2_CFP@kt/nuf2_CFP@cyto']
    df_dmso['condition']='dmso'
    df_dmso['cell']=FOLDER+"_"+'dmso_cell_'+str(cell_count)
    expt_name=re.sub(r'^.+/([^/]+)$', r'\1', dmso_csv[i])
    df_dmso['Experiment']= expt_name#'cdki_%d'%i
    fname=FOLDER +'dmso_'+ str(i)
    df_dmso_final=outliers_pp1(df_dmso)
    df_dmso_final.to_csv(BASE_DIR  + 'GraphAnalysis/pp1/'+FOLDER + '/' + fname +'.csv')
    df_graph_dmso=pd.concat([df_graph_dmso,df_dmso_final],ignore_index=True)
    cell_count+=1
df_graph_dmso = df_graph_dmso.drop(columns=["Unnamed: 0"], axis=1)
df_graph_dmso.to_csv(BASE_DIR  + 'GraphAnalysis/pp1/'+FOLDER+'/' +FOLDER+ "_" +'dmso_pp1_graphs'+'.csv') 

In [11]:
df_graph=pd.DataFrame()
df_graph=pd.concat([df_graph,df_graph_cdki,df_graph_dmso],ignore_index=True)
print(len(df_graph_cdki))
print(len(df_graph_dmso))
print(len(df_graph))
df_graph.head()
df_graph.to_csv(BASE_DIR  + 'GraphAnalysis/pp1/'+FOLDER+'/' +FOLDER+ "_" + 'pp1_combined_graphs'+'.csv') 


489
1153
1642
